In [22]:
from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image
import glob
from os import listdir
from os.path import isfile, join
import os
from time import sleep

class Window:
    def __init__(self):
        self.window=tk.Tk()
        self.window.title("Image Viewer")
        self.window.geometry("640x480")
        self.window.resizable(False, False)

        # 메뉴바를 윈도우에 추가한다.
        menubar = tk.Menu(self.window)
        menu_1 = tk.Menu(menubar, tearoff=0)
        menu_2 = tk.Menu(menubar, tearoff=0)
        menu_3 = tk.Menu(menubar, tearoff=0)
        menu_1.bind('<<MenuSelect>>')

        menu_1.add_command(label="Open", command=self.fileSelect)
        menu_2.add_command(label="StartShow", command=self.SlideShow)
        menu_3.add_command(label="NextSlide", command=self.nextSlide)
        
        menubar.add_cascade(label="File", menu=menu_1)
        menubar.add_cascade(label="Start", menu=menu_2)
        menubar.add_cascade(label="Next", menu=menu_3)
        

        self.window.config(menu=menubar)

        # 이미지 라벨을 추가한다.
        self.imgLabel = tk.Label(self.window, width=400, height=400, relief='solid')
        self.imgLabel.pack()

        self.window.mainloop()
    
    def SlideShow(self):
        self.test = CircleLinkedList()        
        for item in self.files_path:
            self.test.append(self.selFile + '/' + item)        
        _tmp = self.test.root.item
        self.test.setCurrent(_tmp)
        self.image = Image.open(self.test.current.item)
        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))
        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.image = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image = self.image)            
            
    def nextSlide(self):    
        self.test.moveNext()
        self.image = Image.open(self.test.current.item)
        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))
        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.image = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image = self.image)
            
            
    
    # 메뉴에서 close 가 선택되었을 때 수행한다.
    def close(self):
        self.window.quit()
        self.window.destroy()

    # 파일을 선택한다.
    def fileSelect(self):
        # 초기 디렉토리를 루트로 지정하고 파일을 선택하면 해당 파일명을 selFile에 입력한다.
        self.selFile = filedialog.askdirectory()
        print(self.selFile)        
        self.files_path = os.listdir(self.selFile)
        print(self.files_path)


        
class Node:
    def __init__(self, item=None):
        self.item = item
        self.link = None
        
class CircleLinkedList:
    def __init__(self):
        self.root = Node()
        self.tail = self.root
        self.current = self.root
        
    def append(self, item):
        # 추가할 새노드를 만든다.
        newNode = Node(item)
        # 루트가 비어 있으면 루트와 테일을 새노드로 지정한다.
        if self.root.item == None:
            self.root = newNode
            self.tail = newNode
            newNode.link = self.root
        else:
        # 루트가 비어 있지 않으면 테일 뒤에 새노드를 추가하고 테일을 업데이트한다.
            _tmp = self.tail.link
            self.tail.link = newNode
            newNode.link = _tmp
            self.tail = newNode

    # 리스트 안에 모든 노드를 프린트 한다. 
    def print(self):
        curNode = self.root
        print(curNode.item)
        while curNode.link != self.root:
            curNode = curNode.link
            print(curNode.item)

    # 리스트에 존재하는 노드 수를 구한다.
    def listSize(self):
        curNode = self.root
        cnt = 1
        while curNode.link != self.root:
            curNode = curNode.link
            cnt += 1
        return cnt

    # 리스트에서 item을 가지는 노드를 current 노드로 지정한다.
    def setCurrent(self,item):
        curNode = self.root
        for i in range(self.listSize()):
            if curNode.item == item:
                self.current = curNode
            else:
                curNode = curNode.link

    # current 노드를 다음 노드로 바꾼다.
    def moveNext(self):
        self.current = self.current.link

    # current 다음에 item을 삽입한다.
    def insert(self, item):
        # 삽입할 새 노드를 만든다.
        newNode = Node(item)
        # current 노드의 링크에 새 노드를 지정하고 새노드의 링크에 current 노드가 연결되어 있던 링크를 지정하여 연결한다.
        _tmp = self.current.link
        self.current.link = newNode
        newNode.link = _tmp
        # 만약, item1이 tail 노드 였다면 새노드를 tail 노드로 재지정한다.
        if self.current == self.tail:
            self.tail = newNode

    # item을 가지는 노드를 삭제한다.
    def delete(self, item):
        delYN = False
        # 루트가 삭제 대상 노드인지 확인한다. 루트를 삭제할 경우, 루트 다음 노드를 루트로 갱신하고 tail 노드의 링크를 갱신된 루트로 재지정한다.
        curNode = self.root
        if curNode.item == item:
            self.root = self.root.link
            self.tail.link = self.root
            delYN = True
        # 루트가 삭제 대상이 아닐 경우, 끝까지 탐색하면서 item이 존재하는 노드를 찾는다.
        # 찾았다면 preNode의 링크에 대상노드 링크를 넘겨준다. 만약, 대상 노드가 tail노드라면 tail 노드를 preNode로 변경한다.
        else:
            while curNode.link != self.root:
                preNode = curNode
                curNode = curNode.link
                if curNode.item == item:
                    preNode.link = curNode.link
                    if curNode == self.tail:
                        self.tail = preNode
                    delYN = True
        # 삭제를 하지 못했으면 error message를 출력한다.
        if delYN == False: print("delete failed")
            
    def getCurrent(self):
        print(self.current.item)
        
        
a = Window()




C:/Users/tony1/Desktop/test
['다운로드 (1).jpg', '다운로드 (2).jpg', '다운로드 (3).jpg', '다운로드.jpg']
